In [61]:
from scipy.io import arff
import os
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.pylab as plb

import sklearn
from sklearn.model_selection import train_test_split

In [62]:
currentDir = os.getcwd()
datasetDir = currentDir + "/OpenSMILE/"
# trainDir = os.path.join(datasetDir, "train")
# testDir = os.path.join(datasetDir, "test")
# validDir = os.path.join(datasetDir, "valid")
# y_dataDir = os.path.join(datasetDir, "y_data")

In [83]:
data = arff.loadarff(os.path.join(datasetDir, 'train_split_Depression_AVEC2017 - Without Unknown.arff'))
df = pd.DataFrame(data[0])

# df.head()
# df.tail()
df.drop(['name'], axis = 1) #0 for row, 1 for column
y = df['PHQ8_Binary'].values.astype(int)

X = df.drop(['name', 'PHQ8_Binary'], axis = 1).values.astype(np.float32)

In [84]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# # Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()   
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [85]:
# Importing random forest training model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [10, 231, 452, 673, 894, 1115, 1336, 1557, 1778, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [86]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=231, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=231, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=231, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=231, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=   0.4s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=231, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=   0.4s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 


[CV] n_estimators=673, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=673, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=False, total=   1.3s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False 


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   28.2s


[CV]  n_estimators=673, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=False, total=   1.3s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=1778, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=10, bootstrap=False, total=   9.4s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False, total=   3.9s
[CV] n_estimators=1557, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False, total=   4.0s
[CV] n_estimators=1557, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimator

[CV] n_estimators=1778, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True 
[CV]  n_estimators=1336, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True, total=   2.4s
[CV] n_estimators=1778, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True 
[CV]  n_estimators=1336, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True, total=   2.5s
[CV] n_estimators=1336, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=False 
[CV]  n_estimators=1336, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=False, total=   2.5s
[CV] n_estimators=1336, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=False 
[CV]  n_estimators=1336, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=False, total=   2.6s
[CV] n_estimators=13

...

[CV] n_estimators=231, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=231, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False, total=   0.4s
[CV] n_estimators=231, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=231, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False, total=   0.4s
[CV] n_estimators=231, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=231, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False, total=   0.4s
[CV] n_estimators=1557, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=None, bootstrap=True 
[CV]  n_estimators=1557, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=None, bootstrap=True, total=   2.9s
[CV] n_estimator

[CV] n_estimators=231, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=100, bootstrap=True 
[CV]  n_estimators=452, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=40, bootstrap=False, total=   0.9s
[CV] n_estimators=231, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=100, bootstrap=True 
[CV]  n_estimators=231, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=100, bootstrap=True, total=   0.4s
[CV] n_estimators=1557, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True 
[CV]  n_estimators=231, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=100, bootstrap=True, total=   0.4s
[CV] n_estimators=1557, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True 
[CV]  n_estimators=673, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False, total=   4.4s
[CV] n_estimators=1557, 

[CV] n_estimators=10, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=100, bootstrap=False 
[CV]  n_estimators=10, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=100, bootstrap=False, total=   0.1s
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True 
[CV]  n_estimators=894, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False, total=   4.1s
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True 
[CV]  n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True, total=   3.7s
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True 
[CV]  n_estimators=894, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False, total=   5.2s
[CV] n_estimators=2000, min_

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.2min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [10, 231, 452, 673, 894, 1115, 1336, 1557, 1778, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [87]:
print(rf_random.best_estimator_)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=894, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)


In [90]:
best_random = rf_random.best_estimator_

bestmodel = best_random.predict(X_test)
score = classifier.score(X_test, y_test)
print(score)

0.55


In [91]:
print (bestmodel)

[0.09273863 0.08538138 0.25817842 0.38873482 0.10212528 0.09645698
 0.50092278 0.06423955 0.24774688 0.17457339 0.29752894 0.13552471
 0.37502215 0.19555454 0.09569085 0.17798773 0.34475867 0.32350365
 0.21727522 0.11848341]


In [52]:
data = arff.loadarff(os.path.join(datasetDir, 'train_split_Depression_AVEC2017 - Without Unknown.arff'))
df = pd.DataFrame(data[0])

# df.head()
# df.tail()
df.drop(['name'], axis = 1) #0 for row, 1 for column
y_train = df['PHQ8_Binary'].values.astype(int)

x_train = df.drop(['name', 'PHQ8_Binary'], axis = 1).values.astype(np.float32)
x_train

array([[ 2.40e-02,  1.33e-04,  2.39e-02, ...,  1.82e+01,  8.40e-03,
         8.86e+00],
       [ 2.54e-02,  1.30e-04,  2.53e-02, ...,  1.75e+01, -4.91e-02,
         1.22e+01],
       [ 1.38e-01,  2.01e-05,  1.38e-01, ...,  2.09e+01, -1.97e-02,
         1.20e+01],
       ...,
       [ 5.70e-03,  0.00e+00,  5.70e-03, ...,  2.02e+01, -6.80e-02,
         9.53e+00],
       [ 5.90e-03,  0.00e+00,  5.90e-03, ...,  1.85e+01, -6.85e-03,
         1.24e+01],
       [ 1.60e-02,  0.00e+00,  1.60e-02, ...,  1.43e+01, -1.36e-01,
         3.31e+01]], dtype=float32)

In [53]:
print(x_train.shape)

(96, 384)


In [54]:
# Splitting the dataset into the Training set and Test set
X_train2, X_test, y_train2, y_test = train_test_split(x_train, y_train, test_size = 0.25, random_state = 0)

In [55]:
print(X_train2.shape)

(72, 384)


In [56]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train2 = sc.fit_transform(X_train2)
X_test = sc.transform(X_test)

In [58]:
# from sklearn.preprocessing import RobustScaler

# scaler = RobustScaler()

# X_train2 = scaler.fit_transform(X_train2)
# X_test = scaler.transform(X_test)

In [48]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear') # 0.625
# classifier = SVC(kernel="sigmoid", gamma=2) # 50%
# classifier = SVC(kernel="poly", C=0.025) # 58.3%
# classifier =SVC(gamma=2, C=1) # 58.3%
classifier.fit(X_train2, y_train2)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [49]:
classifier.predict(X_test)
score = classifier.score(X_test, y_test)
print(score)

0.625


In [59]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier2 = LogisticRegression(random_state = 0)
# classifier.fit(X_train, y_train)
classifier2.fit(X_train2, y_train2.ravel())                                                                                                                                                                   
score = classifier2.score(X_test, y_test)
print("Prediction Score: ", score)

# Predicting the Test set results
y_pred = classifier2.predict(X_test)

Prediction Score:  0.7083333333333334


In [32]:
import pickle
scalerfile = 'scaler.sav'
pickle.dump(sc, open(scalerfile, 'wb'))

In [33]:
filename = 'OpnSmModel.sav'
pickle.dump(classifier2, open(filename, 'wb'))

In [10]:
# def evaluate(model, X_test, y_test):
#     predictions = model.predict(X_test)
#     errors = abs(predictions - y_test)
#     mape = 100 * np.mean(errors / y_test)
#     accuracy = 100 - mape
#     print('Model Performance')
#     print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
#     print('Accuracy = {:0.2f}%.'.format(accuracy))
    
#     return accuracy

In [17]:
# pred = classifier.predict(X_test)
# print(pred)
# err = abs(pred - y_test)
# print(err)

[1 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0]
[0 0 1 1 0 1 0 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0]


In [19]:

score = classifier.score(X_test, y_test)
print("Prediction Score: ", score)

Prediction Score:  0.625


In [21]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.65      0.79      0.71        14
          1       0.57      0.40      0.47        10

avg / total       0.62      0.62      0.61        24



In [22]:
from sklearn.metrics import matthews_corrcoef
print("Correlation Score: ", matthews_corrcoef(y_test, y_pred))

Correlation Score:  0.20143542511732357


In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[11,  3],
       [ 6,  4]])

In [30]:
# print("X=%s, Predicted=%s" % (X_test[0], y_pred[0]))

In [29]:
classifier.predict_proba(X_test)

AttributeError: predict_proba is not available when  probability=False

In [23]:
# # Predicting the Test set results
# # y_pred = classifier.predict(X_test)

# spam = evaluate(classifier, X_test, y_test)

In [6]:
# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, Activation

# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K

# import tensorflow as tf

Using TensorFlow backend.


In [13]:

# np.savetxt("SingleRow2.csv", x_train[9], delimiter=",")

In [12]:
# len(x_train[0])

384

In [18]:
# model = Sequential()
# model.add(Dense(64, input_dim=384, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

# # from keras.models import Sequential 
# # from keras.layers import Dense, Activation 
# # output_dim = 2
# # model = Sequential() 
# # model.add(Dense(output_dim, input_dim=384, activation='softmax')) 
# # batch_size = 128 
# # nb_epoch = 20

# # model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 
# # history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,verbose=1, validation_data=(X_test, Y_test)) 
# # score = model.evaluate(X_test, Y_test, verbose=0) 
# # print('Test score:', score[0]) 
# # print('Test accuracy:', score[1])

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

In [19]:
# model.fit(x_train, y_train,
#           epochs=50,
#           batch_size=128)

Epoch 1/50
96/96 [==============================] - 1s 12ms/step - loss: 5.5040 - acc: 0.6562
Epoch 2/50
96/96 [==============================] - 0s 79us/step - loss: 5.3489 - acc: 0.6667
Epoch 3/50
96/96 [==============================] - 0s 73us/step - loss: 6.9756 - acc: 0.5625
Epoch 4/50
96/96 [==============================] - 0s 103us/step - loss: 4.8507 - acc: 0.6979
Epoch 5/50
96/96 [==============================] - 0s 71us/step - loss: 5.6737 - acc: 0.6458
Epoch 6/50
96/96 [==============================] - 0s 77us/step - loss: 5.1847 - acc: 0.6771
Epoch 7/50
96/96 [==============================] - 0s 67us/step - loss: 7.0096 - acc: 0.5625
Epoch 8/50
96/96 [==============================] - 0s 80us/step - loss: 6.0150 - acc: 0.6250
Epoch 9/50
96/96 [==============================] - 0s 80us/step - loss: 5.3434 - acc: 0.6667
Epoch 10/50
96/96 [==============================] - 0s 90us/step - loss: 5.6792 - acc: 0.6458
Epoch 11/50
96/96 [==============================] - 0s 81

In [20]:
# model.save('DepressionAnalysisModel - OpenSmileData2.h5')

In [23]:
# model.summary()